In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [4]:
!pip install --upgrade transformers

In [8]:
# Импортируем нужные библиотеки
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Загружаем датасет IMDB (предварительно разбитый на train/test)
dataset = load_dataset("imdb")




In [9]:
# Загружаем токенизатор для BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Функция для токенизации текста
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Токенизируем весь датасет
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Загружаем предобученную модель BERT для классификации (2 класса)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Определяем аргументы обучения
training_args = TrainingArguments(
    output_dir="./results",          # Папка для сохранения результатов
    # 'evaluation_strategy' has been replaced with 'eval_strategy' in newer versions.
    eval_strategy="epoch",     # Оценка после каждой эпохи
    per_device_train_batch_size=8,   # Размер батча для обучения
    per_device_eval_batch_size=8,    # Размер батча для валидации
    num_train_epochs=1,              # Количество эпох
    weight_decay=0.01,               # Сглаживание весов
)

# Используем Trainer API
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(1000)),  # Упрощённый поднабор
    eval_dataset=tokenized_datasets["test"].shuffle(seed=42).select(range(500)),
)

# Запускаем fine-tuning
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shakhidana_nitaliyeva (shakhidana_nitaliyeva-kazakh-british-technical-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,0.333557


TrainOutput(global_step=125, training_loss=0.4519457092285156, metrics={'train_runtime': 7519.4618, 'train_samples_per_second': 0.133, 'train_steps_per_second': 0.017, 'total_flos': 263111055360000.0, 'train_loss': 0.4519457092285156, 'epoch': 1.0})